# Tarea 5

* Verificar que el estado de Bob antes de aprender los resultados de la medición de Alice, es equivalente a tener un estado $|0\rangle$ o $|1\rangle$ con igual probabilidad.
* Hacer un script que verifique el la traza parcial de un estado aleatorio, tiene traza uno y es semipositiva. Ponerlo en forma de Travis CI. 

## Primer punto

**Citando el notebook 05_sistemas_abiertos**

Vamos a suponer que el qubit desconocido está en el estado
\begin{equation}
|\psi\rangle = \alpha |0\rangle + \beta |1\rangle.
\end{equation}
El estado de los otros dos qubits es un par de Bell, esto es
\begin{equation}
|\psi\rangle_{AB} = \frac{1}{\sqrt{2}}( |00\rangle + |11\rangle).
\end{equation}
por lo que el estado inicial será
\begin{align*}
|\psi_0\rangle =  \frac{1}{\sqrt{2}}(\alpha |0\rangle + \beta |1\rangle)(|00\rangle + |11\rangle)
= \frac{1}{\sqrt{2}}(\alpha |000\rangle + \alpha |011\rangle + \beta |100\rangle + \beta |111\rangle).
\end{align*}

Después de eso, tendremos que tenemos que hacer un _control not_ (CN). El CN es una operación unitaria que hace un NOT si el qubit de control está encendido. Esto es, va a tener el efecto
\begin{align*}
U_{CN}|00\rangle &= |00\rangle  \\
U_{CN}|01\rangle &= |01\rangle \\
U_{CN}|10\rangle &= |11\rangle \\
U_{CN}|11\rangle &= |01\rangle
\end{align*}
donde el qubit de control es el primero y el target qubit es el segundo. 

Entonces, el efecto sobre el estado sera 
\begin{equation}
|\psi_1\rangle = U_{CN}^{qA} |\psi_0\rangle = 
\frac{1}{\sqrt{2}}(\alpha |000\rangle + \alpha |011\rangle + \beta |110\rangle + \beta |101\rangle).
\end{equation}
Después hacemos el Hadamard gate ...

\begin{align}
|\psi_2\rangle & =\frac12(
\alpha|000\rangle  + \alpha|100\rangle +  \alpha|011\rangle  + \alpha|111\rangle  + \beta|010\rangle  - \beta|110\rangle  + \beta|001\rangle -\beta|101\rangle) \\
&=\frac12(
|00\rangle(\alpha|0\rangle + \beta|1\rangle) +|01\rangle(\alpha|1\rangle + \beta|0\rangle) +|10\rangle(\alpha|0\rangle - \beta|1\rangle) +|11\rangle(\alpha|1\rangle - \beta|0\rangle)
).
\end{align}

El estado del sistema antes de que Alice realiza su medición es:

$$
|\psi\rangle =\frac12(
\alpha|000\rangle  + \alpha|100\rangle +  \alpha|011\rangle  + \alpha|111\rangle  + \beta|010\rangle  - \beta|110\rangle  + \beta|001\rangle -\beta|101\rangle)
$$
Por lo que la matriz densidad del sistema completo es 

\begin{align}
\rho = \frac{1}{4}\bigg(&|00\rangle\langle00|(\alpha|0\rangle+\beta|1\rangle)(\alpha^*\langle0|+\beta^*\langle1|)  \\
+&|01\rangle\langle01|(\alpha|1\rangle+\beta|0\rangle)(\alpha^*\langle1|+\beta^*\langle0|)  \\
+&|10\rangle\langle10|(\alpha|0\rangle-\beta|1\rangle)(\alpha^*\langle0|-\beta^*\langle1|)  \\
+&|11\rangle\langle11|(\alpha|1\rangle-\beta|0\rangle)(\alpha^*\langle1|-\beta^*\langle0|) \bigg)
\end{align}

Como nos interesa como reacciona el sistema de Bob tenemos que "quitar" la información de del sistema de Alice, por lo tanto se tiene que tomar la traza parcial respecto a los primeros dos qubits.

Por lo que la matriz de densidad reducida de Bob es:
\begin{align}
\rho_B = \frac{1}{4}\bigg(&(\alpha|0\rangle+\beta|1\rangle)(\alpha^*\langle0|+\beta^*\langle1|)  \\
+&(\alpha|1\rangle+\beta|0\rangle)(\alpha^*\langle1|+\beta^*\langle0|)  \\
+&(\alpha|0\rangle-\beta|1\rangle)(\alpha^*\langle0|-\beta^*\langle1|)  \\
+&(\alpha|1\rangle-\beta|0\rangle)(\alpha^*\langle1|-\beta^*\langle0|) \bigg)
\end{align}

Que se puede reducir a 

\begin{align}
\rho_B =& \frac{1}{4}\bigg(2(|\alpha|^2+|\beta|^2)|0\rangle\langle0|+2(|\alpha|^2+|\beta|^2)|1\rangle\langle1| \bigg)\\
\rho_B =& \frac{1}{2}\bigg(|0\rangle\langle0|+|1\rangle\langle1| \bigg)
\end{align}


Por lo tanto cualquier medición que realize Bob en sus sistema sin tener información adicional, la que es proporcionada por Alice, es análogo a tener un resultado aleatorio con probabilidad $\frac{1}{2}$. Adicionalmente si Bob aplica cualquier matriz de Pauli al sistema, no adquiere más información ya que éstas son solamente rotaciones sólidas sobre la esfera de Boch.

Para ver que la traza parcial de un estado aleatorio tiene traza uno y es semipositiva se implemento el siguiente código en un archivo .jl que se encuentra en la carpeta https://github.com/diegohdzrjk/Prueba_travis/tree/master/Tarea5 y el resultado de Travis CI se encuentra en https://github.com/diegohdzrjk/Prueba_travis.

In [2]:
using Base.Test
# Vectorizado
"""
Traza parcial
"""
function partial_trace_pure_bipartite_mat(state,dim,system)
    dimtotal=length(state)[1]
    dimcomp=Int(dimtotal/dim)
    if system==1
    psi=reshape(state,(dimcomp,dim))'
        return psi*psi'
        elseif system==2
     psi=reshape(state,(dim,dimcomp))'
        return psi'*psi
    end
end

function random_state(dim=2::Int)
    v=randn(1,dim)+randn(1,dim)im
    v=v/norm(v)
    return v'
end

function traza_parcial_estado(n)
    return trace(partial_trace_pure_bipartite_mat(random_state(n),n,1))
end


function check_traza_parcial(traza)
    if imag(traza)<1e-6 && abs(real(traza)-1.0)<1e-6
        return true
    else
        return false
    end
end


@test check_traza_parcial(traza_parcial_estado(10))